In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.colors import ListedColormap

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn import metrics

from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import nltk
from nltk.stem import WordNetLemmatizer 
from nltk.stem import PorterStemmer
import re
from spacy.lang.en import English

Import Data

In [4]:
data=pd.read_csv('C://Users//pamela.afful//Desktop//KaggleGithubFolder//train.csv')
test=pd.read_csv('C://Users//pamela.afful//Desktop//KaggleGithubFolder//test.csv')

# Text Preprocessing

In [85]:
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
import string

handles=r'@\w+'
generic_urls=r'http[s]?://(?:[A-Za-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9A-Fa-f][0-9A-Fa-f]))+'


'''

Takes in text data and performs the following:
lowers the case of text
removes URLs
Removes non alphanumeric text
Tokenizes
Stems

'''
def preprocess(df_text):
    df_text=df_text.lower()
    df_text= re.sub(generic_urls, '', df_text) # remove URLs
    df_text= re.sub(handles, '', df_text)# remove Twitter handles
    df_text = re.sub('<[^>]*>', '', df_text)# remove none alphanumeric text
    df_text = re.sub("\\s+(in|the|all|for|and|on)\\s+"," _connector_ ",df_text)
    df_text = re.sub('<.*?>+', '', df_text)
    #df_text = re.sub('[%s]' % re.escape(string.punctuation), '', df_text)
    tokenized_list = nltk.word_tokenize(df_text)
    porter_stem = PorterStemmer()#(language='english')
    #lemmatizer=WordNetLemmatizer() 
    stemed_list=[]
    for i in tokenized_list:
        stemed_list.append(porter_stem.stem(i))
    stop_words=set(stopwords.words('english')) 
    cleaned_list=[i for i in stemed_list if i not in stop_words]# remove stop words
    cleaned_list= [i for i in cleaned_list if len(i)>2]
    
    

        
    return ' '.join(cleaned_list)
    




    

In [75]:
data['cleaned message']=data['message'].apply(preprocess)

In [76]:
pd.set_option('display.max_colwidth',-1)
data

C:\Users\pamela.afful\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


,sentiment,message,tweetid,cleaned message
0,1,"PolySciMajor EPA chief doesn't think carbon dioxide is main cause of global warming and.. wait, what!? https://t.co/yeLvcEFXkC via @mashable",625221,polyscimajor epa chief doe n't think carbon dioxide main cause global warming and.. wait via
1,1,It's not like we lack evidence of anthropogenic global warming,126103,like lack evidence anthropogenic global warming
2,2,RT @RawStory: Researchers say we have three years to act on climate change before it’s too late https://t.co/WdT0KdUr2f https://t.co/Z0ANPT…,698562,researcher say three year act _connector_ climate change late
3,1,#TodayinMaker# WIRED : 2016 was a pivotal year in the war on climate change https://t.co/44wOTxTLcD,573736,todayinmaker wired 2016 pivotal year _connector_ war _connector_ climate change
4,1,"RT @SoyNovioDeTodas: It's 2016, and a racist, sexist, climate change denying bigot is leading in the polls. #ElectionNight",466954,2016 _connector_ racist sexist climate change denying bigot leading _connector_ poll electionnight
...,...,...,...,...
15814,1,"RT @ezlusztig: They took down the material on global warming, LGBT rights, and health care. But now they're hocking Melania's QVC. https://…",22001,took _connector_ material _connector_ global warming lgbt right _connector_ health care 're hocking melania qvc http //…
15815,2,RT @washingtonpost: How climate change could be breaking up a 200-million-year-old relationship https://t.co/rPFGvb2pLq,17856,climate change could breaking 200-million-year-old relationship
15816,0,notiven: RT: nytimesworld :What does Trump actually believe about climate change? Rick Perry joins other aides in … https://t.co/0Mp2,384248,notiven nytimesworld doe trump actually believe climate change rick perry join aide _connector_
15817,-1,RT @sara8smiles: Hey liberals the climate change crap is a hoax that ties to #Agenda2030.\nThe Climate is Being Changed byÃ¢â‚¬Â¦,819732,hey liberal _connector_ climate change crap hoax tie agenda2030 _connector_ climate changed byã¢â‚¬â¦


Split Data and Vectorize

In [77]:
X=data['cleaned message']
y=data['sentiment']

vectorizer = CountVectorizer(ngram_range=(1,2),min_df=2,max_df=0.75)
X_vectorized = vectorizer.fit_transform(X)
X_train,X_val,y_train,y_val = train_test_split(X_vectorized,y,test_size=0.30,shuffle=True, random_state=25)

Fit model

In [78]:

lr=LogisticRegression(multi_class='multinomial',max_iter=300)#solver='newton-cg'
lr.fit(X_train,y_train)
y_pred=lr.predict(X_train)
metrics.f1_score(y_train, y_pred,average='macro')

0.9759763762149121

In [79]:
y_pred=lr.predict(X_val)
metrics.f1_score(y_val, y_pred,average='macro')

0.6491916642293568

In [34]:
param_grid= {'solver':['newton-cg', 'liblinear', 'sag', 'saga','lbfgs'],'C':[1,5,8,10,25]}
grid_log = GridSearchCV( LogisticRegression(multi_class='multinomial'), param_grid, scoring='f1_macro',n_jobs=-1)
grid_log.fit(X_train, y_train)
grid_log.best_params_

{'C': 1, 'solver': 'newton-cg'}

# Applying to test data

In [80]:
test1=test.copy()
test1['cleaned message']=test1['message'].apply(preprocess)

In [68]:
test1

,message,tweetid,sentiment,cleaned message
0,Europe will now be looking to China to make sure that it is not alone in fighting climate change… https://t.co/O7T8rCgwDq,169760,1,europ look china make sure alon _connector_ fight climat change…
1,Combine this with the polling of staffers re climate change and womens' rights and you have a fascist state. https://t.co/ifrm7eexpj,35326,1,combin thi _connector_ poll staffer climat chang _connector_ women right _connector_ fascist state
2,"The scary, unimpeachable evidence that climate change is already here: https://t.co/yAedqcV9Ki #itstimetochange #climatechange @ZEROCO2_;..",224985,1,scari unimpeach evid climat chang alreadi itstimetochang climatechang
3,@Karoli @morgfair @OsborneInk @dailykos \nPutin got to you too Jill ! \nTrump doesn't believe in climate change at all \nThinks it's s hoax,476263,1,putin got jill trump doe n't believ _connector_ climat chang _connector_ think hoax
4,RT @FakeWillMoore: 'Female orgasms cause global warming!'\n-Sarcastic Republican,872928,0,'femal orgasm caus global warm -sarcast republican
...,...,...,...,...
10541,"RT @BrittanyBohrer: Brb, writing a poem about climate change. #climatechange #science #poetry #fakenews #alternativefacts https://t.co/RpUs…",895714,1,brb write poem climat chang climatechang scienc poetri fakenew alternativefact
10542,"2016: the year climate change came home: During the hottest year on record, Karl Mathiesen travelled to Tasmania… https://t.co/WwSrJQfvMg",875167,1,2016 _connector_ year climat chang came home dure _connector_ hottest year _connector_ record karl mathiesen travel tasmania…
10543,RT @loop_vanuatu: Pacific countries positive about Fiji leading the global climate change conference in November. https://t.co/PIPRndhkYd,78329,2,pacif countri posit fiji lead _connector_ global climat chang confer _connector_ novemb
10544,"RT @xanria_00018: You’re so hot, you must be the cause for global warming. #ALDUBLaborOfLove @jophie30 @asn585",867455,0,hot must _connector_ caus _connector_ global warm aldublaboroflov


In [81]:
X_test=vectorizer.transform(test1['cleaned message'])
y_test_pred=lr.predict(X_test)

In [82]:
test['sentiment']=y_test_pred

In [83]:
test

,message,tweetid,sentiment
0,Europe will now be looking to China to make sure that it is not alone in fighting climate change… https://t.co/O7T8rCgwDq,169760,1
1,Combine this with the polling of staffers re climate change and womens' rights and you have a fascist state. https://t.co/ifrm7eexpj,35326,1
2,"The scary, unimpeachable evidence that climate change is already here: https://t.co/yAedqcV9Ki #itstimetochange #climatechange @ZEROCO2_;..",224985,1
3,@Karoli @morgfair @OsborneInk @dailykos \nPutin got to you too Jill ! \nTrump doesn't believe in climate change at all \nThinks it's s hoax,476263,1
4,RT @FakeWillMoore: 'Female orgasms cause global warming!'\n-Sarcastic Republican,872928,0
...,...,...,...
10541,"RT @BrittanyBohrer: Brb, writing a poem about climate change. #climatechange #science #poetry #fakenews #alternativefacts https://t.co/RpUs…",895714,1
10542,"2016: the year climate change came home: During the hottest year on record, Karl Mathiesen travelled to Tasmania… https://t.co/WwSrJQfvMg",875167,1
10543,RT @loop_vanuatu: Pacific countries positive about Fiji leading the global climate change conference in November. https://t.co/PIPRndhkYd,78329,2
10544,"RT @xanria_00018: You’re so hot, you must be the cause for global warming. #ALDUBLaborOfLove @jophie30 @asn585",867455,0


In [84]:
test[['tweetid','sentiment']].to_csv('testsubmission20.csv', index=False)